<a href="https://colab.research.google.com/github/daniel-of-sandwich/Data_Mining_Project/blob/main/csc_426_501_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Daniel Forbes --
Professor Ables --
CSC-426-501 --
12 December 2024 \
Final Project

# Preface

**Data:** \
* U.S. Centers for Disease Control and Prevention (CDC) -- 2023 BRFSS Data (SAS Transport Format) https://www.cdc.gov/brfss/annual_data/annual_2023.html
* U.S. Centers for Disease Control and Prevention (CDC) -- 2022 BRFSS Data (SAS Transport Format) https://www.cdc.gov/brfss/annual_data/annual_2022.html
* U.S. Census Bureau -- Annual Population Estimates, Estimated Components of Resident Population Change, and Rates of the Components of Resident Population Change for the United States, States, District of Columbia, and Puerto Rico: April 1, 2020 to July 1, 2023 (NST-EST2023-ALLDATA) https://www.census.gov/data/tables/time-series/demo/popest/2020s-state-total.html
* U.S. Census Bureau -- 2023 U.S. state map data (cb_2023_us_state_500k) https://www2.census.gov/geo/tiger/GENZ2023/shp

**Resources used:** \
* Python Standard Library documentation https://docs.python.org/3/library/
* Stack Overflow -- Unzipping files in Python https://stackoverflow.com/questions/3451111/unzipping-files-in-python
* Statalist -- Importing problem BRFSS SAS file post 2014 https://www.statalist.org/forums/forum/general-stata-discussion/general/1621644-importing-problem-brfss-sas-file-post-2014
* Stack Overflow -- How to rename a file using Python https://stackoverflow.com/questions/2491222/how-to-rename-a-file-using-python
* 2023 BRFSS Codebook CDC https://www.cdc.gov/brfss/annual_data/annual_2023.html
* NST-EST2023-ALLDATA File Layout https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2020-2023/NST-EST2023-ALLDATA.pdf
* Calculated Variables in Data Files CDC https://www.cdc.gov/brfss/annual_data/2023/pdf/2023-calculated-variables-version4-508.pdf
* ChatGPT https://chatgpt.com/ Note: see "**Use of generative AI (ChatGPT)**" below
* Stack Overflow -- Drop rows on multiple conditions in pandas dataframe https://stackoverflow.com/questions/52456874/drop-rows-on-multiple-conditions-in-pandas-dataframe
* Stack Overflow -- How to predict new data with a trained neural network (Tensorflow 2.0, regression analysis)? https://stackoverflow.com/questions/58939031/how-to-predict-new-data-with-a-trained-neural-network-tensorflow-2-0-regressio
* Classroom_Examples_Teacher_Edition.ipynb on Canvas
* How to Create Data Maps of the United States With Matplotlib https://dev.to/oscarleo/how-to-create-data-maps-of-the-united-states-with-matplotlib-p9i
* Matplotlib user guides https://matplotlib.org/stable/users/index

**Use of generative AI (ChatGPT):**
* Creation of a state FIPS code --> state name dict (state_fips_to_name) to save time.

# Installs and imports

In [ ]:
# Install wget for donwloading datasets from their respective websites
!pip install wget

In [ ]:
# Files and directories
from os import listdir, rename, mkdir, chdir
from os.path import exists, isdir
from wget import download
import zipfile

# Dataset stuff
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Model stuff
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Visualization stuff
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
from PIL import Image
from matplotlib.patches import Patch, Circle
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable

# Download datasets, read each one into a pandas DataFrame

In [ ]:
# CDC 2023 BRFSS Data dataset

# Check if already in /content/
filename_in_directory = False
dataset_xpt_path = ''
print('XPT and csv files currently in /content/:')
for filename in listdir('/content'):
  if filename.endswith('.XPT') or filename.endswith('.csv'):
    print('\t' + filename)
    if filename == 'LLCP2023.XPT':
      filename_in_directory = True
      dataset_xpt_path = '/content/' + filename

# If not already downloaded, download
if not filename_in_directory:
  print('\nDownloading dataset...')

  # Download zipped dataset
  dataset_zip_link = 'https://www.cdc.gov/brfss/annual_data/2023/files/LLCP2023XPT.zip'
  dataset_zip_filename = download(dataset_zip_link)
  dataset_zip_path = '/content/' + dataset_zip_filename

  # Unzip dataset
  dataset_xpt_filename = 'LLCP2023.XPT ' # Space at the end
  dataset_xpt_path = '/content/' + dataset_xpt_filename
  with zipfile.ZipFile(dataset_zip_path, mode='r') as z:
    z.extract(dataset_xpt_filename, path='/content')

  # Remove invisible non-graphic character from end of XPT file extension. This
  #   is a common problem when working with BRFSS datasets apparently
  rename(dataset_xpt_path, dataset_xpt_path[:-1])
  dataset_xpt_filename = dataset_xpt_filename[:-1]
  dataset_xpt_path = dataset_xpt_path[:-1]

  print('Dataset downloaded')
else:
  print('\nDataset not downloaded, already present')

# Read XPT file into a pandas DataFrame. This takes 1-2 minutes
brfss_2023_df = pd.read_sas(dataset_xpt_path)

# If write_csv = True, write df to a csv file so I can read it on LibreOffice
write_csv = False
if write_csv:
  dataset_csv_filename = 'LLCP2023.csv'
  brfss_2023_df.to_csv(dataset_csv_filename, index=False)

In [ ]:
print(brfss_2023_df.shape)

In [ ]:
# CDC 2022 (not 2023) BRFSS Data dataset

# Check if already in /content/
filename_in_directory = False
dataset_xpt_path = ''
print('XPT and csv files currently in /content/:')
for filename in listdir('/content'):
  if filename.endswith('.XPT') or filename.endswith('.csv'):
    print('\t' + filename)
    if filename == 'LLCP2022.XPT':
      filename_in_directory = True
      dataset_xpt_path = '/content/' + filename

# If not already downloaded, download
if not filename_in_directory:
  print('\nDownloading dataset...')
  # Download zipped dataset
  dataset_zip_link = 'https://www.cdc.gov/brfss/annual_data/2022/files/LLCP2022XPT.zip'
  dataset_zip_filename = download(dataset_zip_link)
  dataset_zip_path = '/content/' + dataset_zip_filename

  # Unzip dataset
  dataset_xpt_filename = 'LLCP2022.XPT ' # Space at the end
  dataset_xpt_path = '/content/' + dataset_xpt_filename
  with zipfile.ZipFile(dataset_zip_path, mode='r') as z:
    z.extract(dataset_xpt_filename, path='/content')

  # Remove invisible non-graphic character from end of XPT file extension. This
  #   is a common problem when working with BRFSS datasets apparently
  rename(dataset_xpt_path, dataset_xpt_path[:-1])
  dataset_xpt_filename = dataset_xpt_filename[:-1]
  dataset_xpt_path = dataset_xpt_path[:-1]

  print('Dataset downloaded')
else:
  print('\nDataset not downloaded, already present')

# Read XPT file into a pandas DataFrame. This takes 1-2 minutes
brfss_2022_df = pd.read_sas(dataset_xpt_path)

# If write_csv = True, write df to a csv file so I can read it on LibreOffice
write_csv = False
if write_csv:
  dataset_csv_filename = 'LLCP2022.csv'
  brfss_2022_df.to_csv(dataset_csv_filename, index=False)

In [ ]:
# U.S. Census Bureau 2020-est2023 United States Population dataset

# Check if already in /content/
filename_in_directory = False
dataset_path = ''
print('XPT and csv files currently in /content/:')
for filename in listdir('/content'):
  if filename.endswith('.XPT') or filename.endswith('.csv'):
    print('\t' + filename)
    if filename == 'NST-EST2023-ALLDATA.csv':
      filename_in_directory = True
      dataset_path = '/content/' + filename

# If not already downloaded, download
if not filename_in_directory:
  print('\nDownloading dataset...')

  # Download dataset
  dataset_link = 'https://www2.census.gov/programs-surveys/popest/datasets/2020-2023/state/totals/NST-EST2023-ALLDATA.csv'
  dataset_filename = download(dataset_link)
  dataset_path = '/content/' + dataset_filename

  print('Dataset downloaded')
else:
  print('\nDataset not downloaded, already present')

# Read csv file into a pandas DataFrame
census_df = pd.read_csv(dataset_path)

In [ ]:
# Final print of datasets in /content/
print('XPT and csv files currently in /content/:')
for filename in listdir('/content'):
  if filename.endswith('.XPT') or filename.endswith('.csv'):
    print('\t' + filename)

# Extract useful features from datasets

If sample_size = 1, takes 81 minutes

Dataset feature guide:
* _STATE -- State FIPS Code, calculated variable. Includes territories. 1 (Alabama) to 78 (Virgin Islands). Not alphabetical nor continuous (e.g. 2 is followed by 4), use a dict. Note: Kentucky and Pennsylvania did not have enough data to be included in the 2023 BRFSS dataset. American Samoa and Northern Mariana Islands did not participate.
* STATE -- State FIPS code like _STATE. This one is from the 2023 Census dataset.
* LSATISFY -- Satisfaction with life. 1 (Very satisfied) to 4 (Very dissatisfied). 7 (Don't know/Not sure), 9 (Refused), and BLANK (Not asked or Missing) are ignored. Some states (e.g. Florida) didn't get this question so their LSATISFY value will be NaN.
* EDUCA -- Education Level. 1 (Never attended school or only kindergarten) to 6 (College 4 years or more (College graduate)). 9 (Refused) and BLANK (Not asked or Missing) are ignored.
* INCOME3 -- Income Level. 1 (Less than \$10,000) to 11 (\$200,000 or more). 77 (Don't know/Not sure), 99 (Refused), and BLANK (Not asked or Missing) are ignored.
* _METSTAT -- Metropolitan Status, calculated variable. 1 (Metropolitan counties) or 2 (Nonmetropolitan counties). BLANK (Not defined or Missing) is ignored.
* POPESTIMATE2023 -- 7/1/2023 resident total population estimate.
* NPOPCHG_2023 -- Numeric change in resident total population 7/1/2022 to 7/1/2023.

In [ ]:
# Dict to find the state for a given FIPS code.
state_fips_to_name = {
  1: "Alabama",
  2: "Alaska",
  4: "Arizona",
  5: "Arkansas",
  6: "California",
  8: "Colorado",
  9: "Connecticut",
  10: "Delaware",
  11: "District of Columbia",
  12: "Florida",
  13: "Georgia",
  15: "Hawaii",
  16: "Idaho",
  17: "Illinois",
  18: "Indiana",
  19: "Iowa",
  20: "Kansas",
  21: "Kentucky",
  22: "Louisiana",
  23: "Maine",
  24: "Maryland",
  25: "Massachusetts",
  26: "Michigan",
  27: "Minnesota",
  28: "Mississippi",
  29: "Missouri",
  30: "Montana",
  31: "Nebraska",
  32: "Nevada",
  33: "New Hampshire",
  34: "New Jersey",
  35: "New Mexico",
  36: "New York",
  37: "North Carolina",
  38: "North Dakota",
  39: "Ohio",
  40: "Oklahoma",
  41: "Oregon",
  42: "Pennsylvania",
  44: "Rhode Island",
  45: "South Carolina",
  46: "South Dakota",
  47: "Tennessee",
  48: "Texas",
  49: "Utah",
  50: "Vermont",
  51: "Virginia",
  53: "Washington",
  54: "West Virginia",
  55: "Wisconsin",
  56: "Wyoming",
  60: "American Samoa",
  66: "Guam",
  69: "Northern Mariana Islands",
  72: "Puerto Rico",
  78: "Virgin Islands"
}

# Create a new DataFrame to calculate and store feature data per state. _STATE
#   column initialized with state FIPS codes, other columns initialized to 0.
state_df = pd.DataFrame(data=state_fips_to_name.keys(), columns=['_STATE'])
state_df[['LSATISFY_sum', 'LSATISFY_size', 'LSATISFY_avg', 'EDUCA_sum', \
  'EDUCA_size', 'EDUCA_avg', 'INCOME3_sum', 'INCOME3_size', 'INCOME3_avg', \
  'INCOME3_prev_sum', 'INCOME3_prev_size', 'INCOME3_prev_avg', \
  'income_avg_growth', '_METSTAT_sum', '_METSTAT_size', '_METSTAT_avg']] = 0
state_df[['POPESTIMATE2023', 'NPOPCHG_2023']] = np.nan

# Create a smaller sample of the BFRSS 2023 and 2022 datasets, the full ones
#   take lots of RAM and time
sample_size = 0.02
sample_brfss_2023_bf = pd.DataFrame()
sample_brfss_2022_bf = pd.DataFrame()
if sample_size > 0 and sample_size < 1:
  # BFRSS 2023
  sample_brfss_2023_df = train_test_split(brfss_2023_df[['_STATE', 'LSATISFY', \
    'EDUCA', 'INCOME3', '_METSTAT']], train_size=sample_size, random_state=42)[0]
  # BFRSS 2022 (not 2023)
  sample_brfss_2022_df = train_test_split(brfss_2022_df[['_STATE', 'INCOME3']], \
    train_size=sample_size, random_state=42)[0]
else:
  # BFRSS 2023
  sample_brfss_2023_df = brfss_2023_df[['_STATE', 'LSATISFY', 'EDUCA', \
    'INCOME3', '_METSTAT']]
  # BFRSS 2022 (not 2023)
  sample_brfss_2022_df = brfss_2023_df[['_STATE', 'INCOME3']]

# Iterate through the BRFSS 2023 DataFrame rows and extract desired feature
#   data.
for index, row in sample_brfss_2023_df.iterrows():
  a = row['LSATISFY']
  if a not in [7, 9] and np.isnan(a) == False:
    state_df.loc[state_df['_STATE'] == row['_STATE'], 'LSATISFY_sum'] += a
    state_df.loc[state_df['_STATE'] == row['_STATE'], 'LSATISFY_size'] += 1

  b = row['EDUCA']
  if b != 9 and np.isnan(b) == False:
    state_df.loc[state_df['_STATE'] == row['_STATE'], 'EDUCA_sum'] += b
    state_df.loc[state_df['_STATE'] == row['_STATE'], 'EDUCA_size'] += 1

  c = row['INCOME3']
  if c not in [77, 99] and np.isnan(c) == False:
    state_df.loc[state_df['_STATE'] == row['_STATE'], 'INCOME3_sum'] += c
    state_df.loc[state_df['_STATE'] == row['_STATE'], 'INCOME3_size'] += 1

  d = row['_METSTAT']
  if np.isnan(d) == False:
    state_df.loc[state_df['_STATE'] == row['_STATE'], '_METSTAT_sum'] += d
    state_df.loc[state_df['_STATE'] == row['_STATE'], '_METSTAT_size'] += 1

# Free memory
del brfss_2023_df

# Iterate through the BRFSS 2022 (not 2023) DataFrame rows and extract desired
#   feature data.
for index, row in sample_brfss_2022_df.iterrows():
  c = row['INCOME3']
  if c not in [77, 99] and np.isnan(c) == False:
    state_df.loc[state_df['_STATE'] == row['_STATE'], 'INCOME3_prev_sum'] += c
    state_df.loc[state_df['_STATE'] == row['_STATE'], 'INCOME3_prev_size'] += 1

# Free memory
del brfss_2022_df

# Iterate through the 2023 census DataFrame rows and extract desired feature
#   data.
for index, row in census_df.iterrows():
  if row['STATE'] in state_fips_to_name:
    e = row['POPESTIMATE2023']
    state_df.loc[state_df['_STATE'] == row['STATE'], 'POPESTIMATE2023'] = e

    f = row['NPOPCHG_2023']
    state_df.loc[state_df['_STATE'] == row['STATE'], 'NPOPCHG_2023'] = f

# Calculate averages -- LSATISFY_avg, EDUCA_avg, INCOME3_avg, INCOME3_prev_avg,
#   _METSTAT_avg
for index, row in state_df.iterrows():
  num = row['LSATISFY_sum']
  size = row['LSATISFY_size']
  state_df.loc[state_df['_STATE'] == row['_STATE'], 'LSATISFY_avg'] = \
    float(num) / float(size) if size != 0 else np.nan

  num = row['EDUCA_sum']
  size = row['EDUCA_size']
  state_df.loc[state_df['_STATE'] == row['_STATE'], 'EDUCA_avg'] = \
  float(num) / float(size) if size != 0 else np.nan

  num = row['INCOME3_sum']
  size = row['INCOME3_size']
  state_df.loc[state_df['_STATE'] == row['_STATE'], 'INCOME3_avg'] = \
  float(num) / float(size) if size != 0 else np.nan

  num = row['INCOME3_prev_sum']
  size = row['INCOME3_prev_size']
  state_df.loc[state_df['_STATE'] == row['_STATE'], 'INCOME3_prev_avg'] = \
  float(num) / float(size) if size != 0 else np.nan

  num = row['_METSTAT_sum']
  size = row['_METSTAT_size']
  state_df.loc[state_df['_STATE'] == row['_STATE'], '_METSTAT_avg'] = \
  float(num) / float(size) if size != 0 else np.nan

# Calculate income_avg_growth, manually set _METSTAT_avg for certain states to
#   1 (no metropolitan areas)
no_metro_fips = {66, 72, 78}
for index, row in state_df.iterrows():
  inc23 = row['INCOME3_avg']
  inc22 = row['INCOME3_prev_avg']
  state_df.loc[state_df['_STATE'] == row['_STATE'], 'income_avg_growth'] = \
  inc23 - inc22 if inc23 != 0 and inc22 != 0 else np.nan

  state_df.loc[state_df['_STATE'] == row['_STATE'], '_METSTAT_avg'] = 1 if \
    row['_STATE'] in no_metro_fips else row['_METSTAT_avg']

# If write_csv = True, write df to csv file so I can read it on LibreOffice
write_csv = True
if write_csv:
  dataset_csv_filename = 'state_df.csv'
  state_df.to_csv(dataset_csv_filename, index=False)

# Create and use Feed-Forward Neural Network class to predict missing LSATISFY_avg values

In [ ]:
# A feed-forward neural network. Input DataFrame and target label list. Has a
#   method to predict target values.
class FFNN:
  def __init__(self, df, target_label_list):
    # Drop rows with missing values
    df = df.dropna(axis=0, how='any')

    # Reset index after dropping rows. Doesn't actually affect anything, just
    #   for readability.
    df = df.reset_index(drop=True)

    # Save pre-normalized feature metrics
    self.describe = df.describe()

    # Scale non-target features to fit [0,1]
    self.scaler_X = MinMaxScaler()
    df_scaled = self.scaler_X.fit_transform(df.drop(target_label_list, axis=1))
    df_X = pd.DataFrame(df_scaled, columns=df.drop(target_label_list, axis=1).columns)

    # Scale target features to fit [0,1]
    self.scaler_y = MinMaxScaler()
    df_target_scaled = self.scaler_y.fit_transform(df[target_label_list])
    df_y = pd.DataFrame(df_target_scaled, columns=df[target_label_list].columns)
    df = pd.concat([df_y, df_X], axis=1)

    # Split dataset into train and test sets
    X = df.drop(target_label_list, axis=1)
    y = df[target_label_list]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, \
      random_state=42)

    # Split train set into train and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, \
      test_size=0.2, random_state=42)
    print('X_train.shape = {}\ny_train.shape = {}'.format(X_train.shape, y_train.shape))

    # Model characteristics
    total_epochs = 100
    try: # If shape is (int,int)
      input_shape = X_train.shape[1]
    except IndexError: # If shape is (int,)
      input_shape = 1
    try: # If shape is (int,int)
      output_shape = y_train.shape[1]
    except IndexError: # If shape is (int,)
      output_shape = 1
    hidden_nodes = X_train.shape[1]
    activation = 'relu'
    early_stopping = EarlyStopping(monitor='mse', patience = 5, verbose = 1)

    # Initialize the model and its layers
    self.model = Sequential([
        # Input layer
        Input(shape=(input_shape,)),
        # Dense layer
        Dense(units=hidden_nodes * 2 ** 3, activation=activation),
        # Dense layer
        Dense(units=hidden_nodes * 2 ** 2, activation=activation),
        # Output layer
        Dense(units=output_shape, activation='sigmoid')
    ])

    # Compile the model
    self.model.compile(loss='mse', optimizer='adam', metrics=['mse', 'mae'])

    # Train the model
    history = self.model.fit(X_train, y_train, validation_data=(X_val, y_val), \
      epochs=total_epochs, callbacks=[early_stopping], verbose=0)

    # Get predictions
    y_predictions = self.model.predict(X_test)

    # Test the model
    loss, mse, mae = self.model.evaluate(X_test, y_test)
    y_pred = np.argmax(y_predictions, axis=1)

    # Print model statistics
    print('\nFeed-Forward Neural Network\n')
    print('Epochs: {}'.format(len(history.epoch)))
    print(f'MeanSquarredError (Loss): {loss:.4f}')
    print(f'MeanAbsoluteError: {mae:.4f}')

  # Input df sans LSATISFY_avg feature, return predicted LSATISFY_arg feature
  def predict(self, X):
    # Scale input df
    X_scaled = pd.DataFrame(self.scaler_X.transform(X), columns=X.columns)
    # Predict target from scaled df
    pred = self.model.predict(X_scaled, verbose=0)
    # Inverse scale predicted target
    pred_scaled = pd.DataFrame(self.scaler_y.inverse_transform(pred))

    return pred_scaled

# Create DataFrame df from state_df that excludes the features used for
#   calculating averages, e.g. sum and size features
df = state_df[['_STATE', 'LSATISFY_avg', 'EDUCA_avg', 'INCOME3_avg', \
  'income_avg_growth', '_METSTAT_avg', 'POPESTIMATE2023', 'NPOPCHG_2023']]

# Drop rows with missing values, not including LSATISFY_avg
df = df.dropna(axis=0, how='any', subset=df.drop('LSATISFY_avg', axis=1).columns)
df = df.reset_index(drop=True)

# Create model from df. The model will predict state-average reported life
#   satisfaction (LSATISFY_avg).
target_label_list = pd.Series(['LSATISFY_avg'])
model = FFNN(df.drop('_STATE', axis=1), target_label_list)

# Use the model to predict missing LSATISFY_avg values
num = 0
for index, row in df.iterrows():
  if np.isnan(row['LSATISFY_avg']):
    pred = model.predict(df.loc[df['_STATE'] == row['_STATE']].drop(['_STATE', \
      'LSATISFY_avg'], axis=1))[0][0]
    df.loc[df['_STATE'] == row['_STATE'], 'LSATISFY_avg'] = pred

# If write_csv = True, write df's to csv files so I can read them on LibreOffice
write_csv = True
if write_csv:
  dataset_csv_filename = 'df.csv'
  df.to_csv(dataset_csv_filename, index=False)

# Download map data

In [ ]:
# 2023 Census Map Data dataset

# Check if already in /content/
new_dir_already_exists = False
new_dir_path = ''
print('directories currently in /content/:')
for dir in listdir('/content'):
  if isdir(dir):
    print('\t' + dir)
    if dir == 'cb_2023_us_state_500k':
      new_dir_already_exists = True
      new_dir_path = '/content/' + dir

# If not already downloaded, download
if not new_dir_already_exists:
  print('\nDownloading zip...')

  # Create new directory
  new_dir = 'cb_2023_us_state_500k'
  mkdir(new_dir)
  new_dir_path = '/content/' + new_dir

  # Download zipped dataset
  zip_link = 'https://www2.census.gov/geo/tiger/GENZ2023/shp/cb_2023_us_state_500k.zip'
  chdir(new_dir_path)
  zip_filename = download(zip_link)
  chdir('..')
  zip_path = new_dir_path + '/' + zip_filename

  # Unzip dataset
  with zipfile.ZipFile(zip_path, mode='r') as z:
    z.extractall(path=new_dir_path)

  print('Dataset downloaded')
else:
  print('\nzip not downloaded, already present')

# Create GeoDataFrame from DataFrame for plotting

In [ ]:
# Fn to adjust df geometry
def translate_geometries(df, x, y, scale, rotate):
    df.loc[:, "geometry"] = df.geometry.translate(yoff=y, xoff=x)
    center = df.dissolve().centroid.iloc[0]
    df.loc[:, "geometry"] = df.geometry.scale(xfact=scale, yfact=scale, \
      origin=center)
    df.loc[:, "geometry"] = df.geometry.rotate(rotate, origin=center)

    return df

# Fn to move Alaska and Hawaii closer, shrink Alaska
def adjust_maps(df):
    df_main_land = df[~df.STATEFP.isin(["02", "15"])]
    df_alaska = df[df.STATEFP == "02"]
    df_hawaii = df[df.STATEFP == "15"]

    df_alaska = translate_geometries(df_alaska, 1300000, -4900000, 0.5, 32)
    df_hawaii = translate_geometries(df_hawaii, 5400000, -1500000, 1, 24)

    return pd.concat([df_main_land, df_alaska, df_hawaii])

# Load and prepare geo-data, excludes the unincorporated territories
states = gpd.read_file("/content/cb_2023_us_state_500k/")
states = states[~states.STATEFP.isin(["72", "69", "60", "66", "78"])]
states = states.to_crs("ESRI:102003")
states = adjust_maps(states)

# Add FIPS codes to states
state_name_to_fips = dict(zip(state_fips_to_name.values(), \
  state_fips_to_name.keys()))
states['_STATE'] = states['NAME'].map(state_name_to_fips)

# Add LSATISFY_avg to states
states['LSATISFY_avg'] = df['LSATISFY_avg'].describe()['mean']
for index, row in states.iterrows():
  if len(df.loc[df['_STATE'] == row['_STATE'], 'LSATISFY_avg']) > 0:
    a = df.loc[df['_STATE'] == row['_STATE'], 'LSATISFY_avg'].iloc[0]
  else:
    a = df['LSATISFY_avg'].describe()['mean']
  states.loc[states['_STATE'] == row['_STATE'], 'LSATISFY_avg'] = a

# Add POPESTIMATE2023 to states
states['POPESTIMATE2023'] = 0
for index, row in states.iterrows():
  if len(census_df.loc[census_df['STATE'] == row['_STATE'], 'POPESTIMATE2023']) > 0:
    a = census_df.loc[census_df['STATE'] == row['_STATE'], 'POPESTIMATE2023'].iloc[0]
  else:
    a = 0
  states.loc[states['_STATE'] == row['_STATE'], 'POPESTIMATE2023'] = a

# Plot reported life satisfaction map

In [ ]:
# Prepare Seaborn
edge_color = "#30011E"
background_color = "#fafafa"
sns.set_style({
    "font.family": "serif",
    "figure.facecolor": background_color,
    "axes.facecolor": background_color,
})

# Set global font style
sns.set_context('notebook', 1.5)

# Create a color palette
palette = sns.color_palette('coolwarm', as_cmap=True)

# Normalize FIPS codes for coloring
norm = Normalize(vmin=states['LSATISFY_avg'].describe()['min'], \
  vmax=states['LSATISFY_avg'].describe()['max'])

# Create the plot
fig, ax = plt.subplots(1, 1, figsize=(20, 20))
states.boundary.plot(ax=ax, linewidth=1)
states.plot(column='LSATISFY_avg', cmap=palette, legend=False, ax=ax)

# Add a title
plt.title("2023 U.S. State Self-Reported Life Satisfaction")

# Add a colorbar
sm = ScalarMappable(cmap=palette, norm=norm)
sm._A = []  # Necessary for creating a colorbar
cbar = fig.colorbar(sm, ax=ax, orientation='horizontal', pad=0.05)
cbar.set_label('State average of BRFSS LSATISFY,\nlower (blue) is more satisfied')

# Draw the plot
plt.axis('off')
plt.show()

# Plot population map

In [ ]:
# Prepare Seaborn
edge_color = "#30011E"
background_color = "#fafafa"
sns.set_style({
    "font.family": "serif",
    "figure.facecolor": background_color,
    "axes.facecolor": background_color,
})

# Set global font style
sns.set_context('notebook', 1.5)

# Create a color palette
#palette = sns.color_palette('flare', as_cmap=True)
palette = sns.color_palette("YlOrBr", as_cmap=True)

# Normalize FIPS codes for coloring
norm = Normalize(vmin=states['POPESTIMATE2023'].describe()['min'], \
  vmax=states['POPESTIMATE2023'].describe()['max'])

# Create the plot
fig, ax = plt.subplots(1, 1, figsize=(20, 20))
states.boundary.plot(ax=ax, linewidth=1)
states.plot(column='POPESTIMATE2023', cmap=palette, legend=False, ax=ax)

# Add a title
plt.title("2023 U.S. State Population")

# Add a colorbar
sm = ScalarMappable(cmap=palette, norm=norm)
sm._A = []  # Necessary for creating a colorbar
cbar = fig.colorbar(sm, ax=ax, orientation='horizontal', pad=0.05)
cbar.set_label('State population')

# Draw the plot
plt.axis('off')
plt.show()

# End